In [ ]:
!pip install git+https://github.com/marzbana/GARS.git

  Cloning https://github.com/marzbana/GARS.git to /tmp/pip-req-build-9645ccxu
  Running command git clone --filter=blob:none --quiet https://github.com/marzbana/GARS.git /tmp/pip-req-build-9645ccxu
  Resolved https://github.com/marzbana/GARS.git to commit 1f799d6160a0713509b5e8be946c4c2053af2d12
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/shubhamugare/mxeval.git to /tmp/pip-install-he_iqnzz/mxeval_1f0362376d3a4937803f48fded73926a
  Running command git clone --filter=blob:none --quiet https://github.com/shubhamugare/mxeval.git /tmp/pip-install-he_iqnzz/mxeval_1f0362376d3a4937803f48fded73926a
  Resolved https://github.com/shubhamugare/mxeval.git to commit 590129d063c5e7d145e8b15a83f00ff20d69a189
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/uiuc-focal-lab/syncode.git to /tmp/pip-install-he_iqnzz/syncode_d8ddd5d8ccb8470aba6b84834abf6b26
  Running command git clone --filter=blob:none --quiet https://github.com/uiuc-focal-lab/syncode.git /tm

In [ ]:
# **1. Import Necessary Libraries**
import torch
from syncode2 import SyncodeLogitsProcessor2
from syncode import Grammar
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList, BitsAndBytesConfig
from datasets import load_dataset
from evaluate import load
import random
import re
from tqdm import tqdm

In [ ]:
# **2. Setup Device**
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# **3. Load the Model**
model_name = 'm-a-p/OpenCodeInterpreter-DS-6.7B'
# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name,padding_side='left')
model = AutoModelForCausalLM.from_pretrained(
       model_name,
       device_map="cuda",
       torch_dtype=torch.bfloat16,
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [ ]:
special_tokens_dict = {}
if tokenizer.pad_token is None:
    special_tokens_dict['pad_token'] = '<pad>'
if tokenizer.eos_token is None:
    special_tokens_dict['eos_token'] = '</s>'

if special_tokens_dict:
    tokenizer.add_special_tokens(special_tokens_dict)
    model.resize_token_embeddings(len(tokenizer))

# Assign pad_token_id and eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id
model.config.eos_token_id = tokenizer.eos_token_id

# Verify assignments
print("Pad token ID:", tokenizer.pad_token_id)  # Should be 0 for LLaMA
print("EOS token ID:", tokenizer.eos_token_id)  # Should be 2 for LLaMA

Pad token ID: 32014
EOS token ID: 32021


In [ ]:
# **4. Initialize SynCode with Python Grammar**
grammar = Grammar(name='python')
syncode_logits_processor = SyncodeLogitsProcessor2(grammar=grammar, tokenizer=tokenizer, parse_output_only=False)


Creating DFA mask store for LlamaTokenizerFast and python, may take more than 10 minutes. Caching at /content/cache/mask_stores/LlamaTokenizerFast/grammar_mask_2631017962_32000.pkl.
Ignore whitespace tokens is True


100%|██████████| 402/402 [13:44<00:00,  2.05s/it]


In [ ]:
# **5. Define Sampling Parameters**
sampling_sizes = [50]
temperature = 0.7  # Adjust as per hyperparameter tuning
max_length = 512  # Adjust based on model capacity and dataset

In [ ]:
# **6. Load Benchmark Dataset**

# Using HumanEval dataset
dataset = load_dataset("openai_humaneval")['test']
dataset_list = list(dataset)

random_problems = random.sample(dataset_list, 20)

README.md:   0%|          | 0.00/6.52k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/83.9k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/164 [00:00<?, ? examples/s]

In [ ]:
# **7. Define Functions for Zero-shot and Two-shot Prompting**

def get_zero_shot_prompt(sample):
    return sample['prompt']

def get_two_shot_prompt(samples, current_sample):
    # Randomly select two samples for few-shot examples
    i = random.randint(0, len(samples) - 1)
    example = samples[i]
    prompt = f"""
    #EXAMPLE:
    {example['prompt']}
    #OUTPUT:
    {example['canonical_solution']}


    #PROBLEM:
    {current_sample['prompt']}
    """
    return prompt

In [ ]:
dataset_samples = [sample for sample in dataset]

In [ ]:
def extract_indented_block(code: str) -> str:
    """
    Extracts a block of text starting from a specified line in the code where
    each subsequent line is indented. Stops when a non-indented line is encountered.

    Parameters:
    - code (str): The full code as a string.
    - start_line_index (int): The index of the line to start extracting from (0-based).

    Returns:
    - str: The indented block of text.
    """
    # Split the code into lines
    lines = code.splitlines()

    # Initialize the indented block
    indented_block = []

    # Iterate through the lines starting from the given index
    for i in range(0, len(lines)):
        line = lines[i]
        if line.strip() == "":
            # Skip empty lines
            continue
        if not line.startswith(" "):  # Stop when a non-indented line is encountered
            break
        indented_block.append(line)

    # Join the indented lines into a single block
    return "\n".join(indented_block)

In [ ]:
def grab_name(function_declaration):
  pattern = r'\bdef\s+([a-zA-Z_][a-zA-Z0-9_]*)\s*\('

  # Extract the function name
  match = re.search(pattern, function_declaration)
  if match:
      return match.group(1)  # Group 1 contains the function name
  else:
      print("No function name found")

In [ ]:
# **8.1 Zero-shot Prompting**
results_zero = [[
    [] for _ in range(len(random_problems))
] for _ in range(len(sampling_sizes))]

test_cases = []

print("Running Zero-shot Prompting...")
# Enumerate to get both index and value from sampling_sizes
for i, num_samples in enumerate(sampling_sizes):
    syncode_logits_processor = SyncodeLogitsProcessor2(grammar=grammar, tokenizer=tokenizer, parse_output_only=False, num_samples=num_samples)
    for num, sample in enumerate(random_problems):
        print(f"Problem {num}")
        prompt1 = get_zero_shot_prompt(sample)
        prompt = [prompt1 for _ in range(num_samples)]
        function_name = grab_name(prompt1)
        test_cases.append(sample['test']+f'\ncheck({function_name})')

        # Tokenize prompt once
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        syncode_logits_processor.reset(prompt)

        # Generate multiple sequences at once
        with torch.no_grad():
            try:
                output = model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_length=512,
                    do_sample=True,
                    temperature=temperature,
                    logits_processor=LogitsProcessorList([syncode_logits_processor]),
                    top_p=0.95,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id
                )
            except Exception as e:
                print(f"An error occurred during generation: {e}")
                output = None

        # Decode all returned sequences
        if output is not None:
            for seq_idx in range(num_samples):
                code3 = tokenizer.decode(output[seq_idx][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
                code3 = extract_indented_block(code3)
                code = prompt1 + '\n' + code3
                # Use i (the index of the sampling size), not num_samples
                results_zero[i][num].append(code)

Running Zero-shot Prompting...
Problem 0
Problem 1
Problem 2
An error occurred during generation: max() arg is an empty sequence
Problem 3
Problem 4
Problem 5
Problem 6
Problem 7
Problem 8
Problem 9
Problem 10
Problem 11
Problem 12
Problem 13
Problem 14
Problem 15
An error occurred during generation: max() arg is an empty sequence
Problem 16
Problem 17
Problem 18
Problem 19


In [ ]:
import os
os.environ["HF_ALLOW_CODE_EVAL"] = "1"

In [ ]:
# Compute pass@k
results2= results_zero[0]

code_eval_metric = load("code_eval")

k_values = [1, 5, 10, 20, 50]
print("Evaluating generated code...")
pass_at_k, results3 = code_eval_metric.compute(
    references=test_cases,
    predictions=results2,
    k=k_values,
    num_workers=4,  # Adjust based on your system
    timeout=10.0,   # Adjust the timeout as needed
)

# Print the results
for k in k_values:
    print(f"Pass@{k}: {pass_at_k[f'pass@{k}'] * 100:.2f}%")

Evaluating generated code...
Pass@1: 50.67%
Pass@5: 77.93%
Pass@10: 86.00%
Pass@20: 91.18%
Pass@50: 94.44%


In [ ]:
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# **8.2 No syncode**
results_no = [[
    [] for _ in range(len(random_problems))
] for _ in range(len(sampling_sizes))]

print("Running Zero-shot Prompting...")
# Enumerate to get both index and value from sampling_sizes
for i, num_samples in enumerate(sampling_sizes):
    for num, sample in enumerate(random_problems):
        print(f"Problem {num}")
        prompt1 = get_zero_shot_prompt(sample)
        prompt = [prompt1 for _ in range(num_samples)]
        function_name = grab_name(prompt1)

        # Tokenize prompt once
        inputs = tokenizer(prompt, return_tensors='pt').to(device)
        # Generate multiple sequences at once
        with torch.no_grad():
            try:
                output = model.generate(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    max_length=512,
                    do_sample=True,
                    temperature=temperature,
                    top_p=0.95,
                    num_return_sequences=1,
                    eos_token_id=tokenizer.eos_token_id,
                    pad_token_id=tokenizer.pad_token_id
                )
            except Exception as e:
                print(f"An error occurred during generation: {e}")
                output = None

        # Decode all returned sequences
        if output is not None:
            for seq_idx in range(num_samples):
                code3 = tokenizer.decode(output[seq_idx][inputs['input_ids'].shape[-1]:], skip_special_tokens=True)
                code3 = extract_indented_block(code3)
                code = prompt1 + '\n' + code3
                # Use i (the index of the sampling size), not num_samples
                results_no[i][num].append(code)
        gc.collect()
        torch.cuda.empty_cache()

Running Zero-shot Prompting...
Problem 0
An error occurred during generation: CUDA out of memory. Tried to allocate 2.01 GiB. GPU 0 has a total capacity of 39.56 GiB of which 1.61 GiB is free. Process 4391 has 37.94 GiB memory in use. Of the allocated memory 30.97 GiB is allocated by PyTorch, and 6.48 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Problem 1
Problem 2
Problem 3
An error occurred during generation: CUDA out of memory. Tried to allocate 190.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 108.81 MiB is free. Process 4391 has 39.45 GiB memory in use. Of the allocated memory 33.45 GiB is allocated by PyTorch, and 5.50 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_A

In [ ]:
# Compute pass@k
results2_no= results_no[0]

code_eval_metric = load("code_eval")

k_values = [1,5,10,20,50]
print("Evaluating generated code...")
pass_at_k, results3 = code_eval_metric.compute(
    references=test_cases,
    predictions=results2_no,
    k=k_values,
    num_workers=4,  # Adjust based on your system
    timeout=10.0,   # Adjust the timeout as needed
)

# Print the results
for k in k_values:
    print(f"Pass@{k}: {pass_at_k[f'pass@{k}'] * 100:.2f}%")

Evaluating generated code...
Pass@1: 68.24%
Pass@5: 89.92%
Pass@10: 93.40%
Pass@20: 94.10%
Pass@50: 94.12%


In [ ]:
!pip install calflops

In [ ]:
from calflops import calculate_flops
batch_size = 1
sequence_length = 128  # Typical length for inference
input_shape = (batch_size, sequence_length)

# Calculate FLOPs, MACs, and Params
flops, macs, params = calculate_flops(model=model,
                                      input_shape=(batch_size,sequence_length),
                                      transformer_tokenizer=tokenizer)
print(f"FLOPs: {flops}, MACs: {macs}, Params: {params}")


------------------------------------- Calculate Flops Results -------------------------------------
Notations:
number of parameters (Params), number of multiply-accumulate operations(MACs),
number of floating-point operations (FLOPs), floating-point operations per second (FLOPS),
fwd FLOPs (model forward propagation FLOPs), bwd FLOPs (model backward propagation FLOPs),
default model backpropagation takes 2.00 times as much computation as forward propagation.

Total Training Params:                                                  6.74 B  
fwd MACs:                                                               845.84 GMACs
fwd FLOPs:                                                              1.69 TFLOPS
fwd+bwd MACs:                                                           2.54 TMACs
fwd+bwd FLOPs:                                                          5.08 TFLOPS

-------------------------------- Detailed Calculated FLOPs Results --------------------------------
Each module cacul

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2905: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(
